In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from os import walk
import ast
import matplotlib.cm as cm
import matplotlib.animation as animation
import time
import pickle

In [2]:
import os
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
sys.path.append(r'E:\course\31\machineLearning\AIforMedicine\re2D3D\raft')
sys.path.append(r'E:\course\31\machineLearning\AIforMedicine\re2D3D\raft\core')

from glob import glob
from PIL import Image
from tqdm import tqdm


In [8]:
from raft.core.raft import RAFT
from raft.core.utils import flow_viz
from raft.core.utils.utils import InputPadder
from raft.config import RAFTConfig

In [9]:
config = RAFTConfig(
    0,
    False,
    False,
    False
)

model = RAFT(config)
model

Namespace(dropout=0, alternate_corr=False, small=False, mixed_precision=False)


F:\Python\Python310\lib\site-packages\torch\nn\modules\conv.py:131: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xe (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  self.weight = Parameter(torch.empty(


RAFT(
  (fnet): BasicEncoder(
    (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (relu1): ReLU(inplace=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=Fa

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

weights_path = '/kaggle/input/raft-pytorch/raft-sintel.pth'
# weights_path = '/kaggle/input/raft-pytorch/raft-things.pth'

ckpt = torch.load(weights_path, map_location=device)
model.to(device)
model.load_state_dict(ckpt)

In [8]:
drr_image_files = glob(r'E:\course\31\machineLearning\AIforMedicine\re2D3D\raft\drr_real\drr\*.png')#return满足条件的list or .jpg看到时候的xray使用什么来保存
xray_image_files = glob(r'E:\course\31\machineLearning\AIforMedicine\re2D3D\raft\drr_real\x-ray\*.png')#return满足条件的list or .jpg看到时候的xray使用什么来保存
drr_image_files = sorted(drr_image_files)#Return a new list containing all items from the iterable in ascending order.
xray_image_files = sorted(xray_image_files)#Return a new list containing all items from the iterable in ascending order.

print(np.shape(drr_image_files))
print(np.shape(xray_image_files))

(10,)
Found 10 images
['E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0016.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0017.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0018.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0019.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0020.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0021.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0022.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0023.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0024.png', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\raft\\demo-frames\\frame_0025.png']


In [ ]:
def load_image(imfile, device):
    img = np.array(Image.open(imfile)).astype(np.uint8)
    img = torch.from_numpy(img).permute(2, 0, 1).float()
    return img[None].to(device)


def viz(img1, img2, flo):# do visualization work
    img1 = img1[0].permute(1,2,0).cpu().numpy()
    img2 = img2[0].permute(1,2,0).cpu().numpy()

    flo_mean = np.mean(flo.cpu().numpy())
    flo_std = np.std(flo.cpu().numpy())

    flo = flo[0].permute(1,2,0).cpu().numpy()

    # map flow to rgb image
    flo = flow_viz.flow_to_image(flo)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 4))
    ax1.set_title('input image1')
    ax1.imshow(img1.astype(int))
    ax2.set_title('input image2')
    ax2.imshow(img2.astype(int))
    ax3.set_title(f'optical flow: mean{flo_mean:.1f}, std{flo_std:.1f}')
    cm = ax3.imshow(flo)
    fig.colorbar(cm)
#    plt.colorbar()
    plt.show()

In [ ]:
model.eval()
n_vis = 3

for file1, file2 in tqdm(zip(drr_image_files[:n_vis], drr_image_files[:n_vis])):#[:n]取出list中的前n个元素
    image1 = load_image(file1, device)
    image2 = load_image(file2, device)

    padder = InputPadder(image1.shape)
    image1, image2 = padder.pad(image1, image2)

    with torch.no_grad():
        flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)

    viz(image1, image2, flow_up)